## **Generating Product Image from Customer Reviews**
### 94844 Gen AI Lab Final Project
#### April 2024

In [2]:
# Import Libraries
import os, io, json, transformers, pinecone, fitz, pypdf, faiss, sqlite3, langchain_community, langchain, openai, math, time 
import requests
from transformers import pipeline
import pandas as pd
import numpy as np
from io import StringIO
from dotenv import load_dotenv
from operator import itemgetter
import logging

from langchain import document_loaders, embeddings
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
#from llama_index.core.node_parser import SentenceSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from pinecone import Pinecone, ServerlessSpec, Pinecone         # vector store

from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    SimpleDirectoryReader
)

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core.schema import TextNode
from llama_index.core.extractors import (
    QuestionsAnsweredExtractor,
    TitleExtractor,
)
from llama_index.core.ingestion import IngestionPipeline
from llama_index.llms.openai import OpenAI

# sentence transformers
from sentence_transformers import SentenceTransformer
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

c:\Users\mjaks\anaconda3\envs\genAIlab_hw2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()

access_endpoint_api_key = os.getenv('access_endpoint_api_key')
openai_key = os.getenv('OPENAI_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')
environment = os.getenv('PINECONE_ENV')

# configure Pinecone client
pinecone_api_key = '4cdd7de3-9866-4385-93cd-df26e52ec9bf'
pc = Pinecone(api_key=pinecone_api_key)

In [5]:
# LOGGER
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s',
                    filename='log/test.log')

## **Part 1: Product Selection and Customer Review Data Collection**
- Select 3 different products from different categories on a digital marketplace (e.g., Amazon).
- Please take into consideration different factors (e.g., product categories, popularity levels) when making your selection.
- Explain the rationale of your choices.
- Collect the corresponding product descriptions (textual content) and customer reviews (textual content) for each product

In [6]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.error import HTTPError, URLError
import time

def get_page(url, headers):
    """
    Get webpage results into a Beautiful Soup object.
    """
    try:
        req = Request(url, headers=headers)
        page = urlopen(req)
        soup = BeautifulSoup(page, "html.parser")
        return soup
    except HTTPError as e:
        print(f"HTTP error: {e.code} {e.reason}")
    except URLError as e:
        print(f"URL error: {e.reason}")
    except Exception as e:
        print(f"General error: {e}")
    return None

def scrape_product_details(base_url, product_id, headers):
    """
    Scrape product description and up to 200 most recent reviews.
    """
    url = f"{base_url}/dp/{product_id}/"
    print("page url:", url)
    soup = get_page(url, headers)
    if not soup:
        return None, []

    # Find product description
    try:
        #description = soup.find_all('div', id='productDescription', class_="a-section a-spacing-small").get_text(strip=True)
        description = soup.find('div', id='feature-bullets').get_text(strip=True)
    except AttributeError:
        description = "No description available"

    reviews = []
    # Adjust the URL for 'Most Recent' reviews and iterate through pages
    review_url = f"{base_url}/product-reviews/{product_id}/"
    print("review url:", review_url)
    for i in range(1, 21):  # Assume there are 10 reviews per page, fetch 20 pages
        url = f"{review_url}?pageNumber={i}&sortBy=helpful"
        soup = get_page(url, headers)
        if not soup:
            continue
        review_divs = soup.find_all('div', {'data-hook': 'review'})
        for div in review_divs:
            try:
                #title = div.find('a', {'data-hook': 'review-title'}).get_text(strip=True)
                body = div.find('span', {'data-hook': 'review-body'}).get_text(strip=True)
                reviews.append(body)
                if len(reviews) >= 300:
                    break
            except AttributeError:
                continue
        if len(reviews) >= 300:
            break
        time.sleep(1)  # Sleep to prevent too frequent requests

    return description, reviews

In [7]:
# Usage example
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
urls = {#"https://www.amazon.com/Samsonite-Omni-Hardside-Spinner-Teal": "B013WFNZRE",
        #"https://www.amazon.com/Hurtle-3-Wheeled-Scooter-Kids-Graffiti": "B08BDGS58Q",
        "https://www.amazon.com/DVQ-ART-Einstein-Painting-Colorful": "B075K5MB7N", 
        #"https://www.amazon.com/DeLonghi-ECAM29084SB-Magnifica-LatteCrema-Espresso": "B0B38KRTV6"
        }

descriptions = []
reviews = []
for base_url, product_id in urls.items():
    description, review = scrape_product_details(base_url, product_id, headers)
    descriptions.append(description)
    reviews.append(review)

products = ["Product A"
            #, "Product B", "Product C"
            ]

# Create dictionary
product_data = {}
for i in range(len(products)):
    product_data[products[i]] = {
        "description": descriptions[i],
        "reviews": reviews[i]
    }
print(product_data)

page url: https://www.amazon.com/DVQ-ART-Einstein-Painting-Colorful/dp/B075K5MB7N/
review url: https://www.amazon.com/DVQ-ART-Einstein-Painting-Colorful/product-reviews/B075K5MB7N/
{'Product A': {'description': 'About this itemGrafitti art decor Einstein wall art painting on canvas contemporary art for living room high definition giclee prints artwork for living room bedroom decoration. This is a good choice for gifts for relatives and friends.is is a good choice for gifts for relatives and friends.A perfect wall decorations paintings for living room, bedroom, kitchen, office, hotel, dining room, office, bar etc..Stretched and Framed canvas art prints ready to hang for home decorations wall decor. Each panel has a black hook already mounted on the wooden bar for easy hanging out of box.HD printing, waterproof, high-quality thick canvas, friendly environment ink jet, high-quality wood and hookHigh Definition Giclee modern canvas printing artwork, picture photo printed on high quality ca

In [8]:
print(product_data['Product A']['description'], '\n', product_data['Product A']['reviews'])

About this itemGrafitti art decor Einstein wall art painting on canvas contemporary art for living room high definition giclee prints artwork for living room bedroom decoration. This is a good choice for gifts for relatives and friends.is is a good choice for gifts for relatives and friends.A perfect wall decorations paintings for living room, bedroom, kitchen, office, hotel, dining room, office, bar etc..Stretched and Framed canvas art prints ready to hang for home decorations wall decor. Each panel has a black hook already mounted on the wooden bar for easy hanging out of box.HD printing, waterproof, high-quality thick canvas, friendly environment ink jet, high-quality wood and hookHigh Definition Giclee modern canvas printing artwork, picture photo printed on high quality canvas. A great gift idea for your relatives and friends. 
 ['Nice bright colors to add a little pop to any room', 'Amazing quality and got here fast! Bigger than I thought too! :)', "I am pleased with this purchas

### Save scraped output

In [9]:
import json

# Write JSON to a file
with open('wall_data.json', 'w', encoding='utf-8') as f:
    json.dump(product_data, f, ensure_ascii=False, indent=4)





## **Part 2: Analysis of Customer Reviews with LLM**
- Use OpenAI’s GPT 3.5 turbo API to conduct text analysis to extract valuable information from the textual data collected above and build a more wholistic understanding about the product.
- Some analyses that are relevant to consider include, but not limited to, for example, text summarization, extraction of particular product features (e.g., visual information), or sentiment analysis.
- You can do the analyses using prompt engineering (with different prompt strategies), RAG, or a combination of both.
- When doing the analysis, you may need to consider different documentation chunking strategies given the input token limit OpenAI API has.
- You could consider using a vector database to store your text embedding if necessary, but not mandatory.
- You also need to think about what is an effective output from this step, given that your goal next step is to send this output into the diffusion model for meaningful product image generation.

In [10]:
# Path to the JSON file
json_file_path = 'wall_data.json'

# Read JSON from a file
with open(json_file_path, 'r', encoding='utf-8') as file:
    product_data = json.load(file)

# Print the data to verify it's read correctly
print(product_data)

{'Product A': {'description': 'About this itemGrafitti art decor Einstein wall art painting on canvas contemporary art for living room high definition giclee prints artwork for living room bedroom decoration. This is a good choice for gifts for relatives and friends.is is a good choice for gifts for relatives and friends.A perfect wall decorations paintings for living room, bedroom, kitchen, office, hotel, dining room, office, bar etc..Stretched and Framed canvas art prints ready to hang for home decorations wall decor. Each panel has a black hook already mounted on the wooden bar for easy hanging out of box.HD printing, waterproof, high-quality thick canvas, friendly environment ink jet, high-quality wood and hookHigh Definition Giclee modern canvas printing artwork, picture photo printed on high quality canvas. A great gift idea for your relatives and friends.', 'reviews': ['Nice bright colors to add a little pop to any room', 'Amazing quality and got here fast! Bigger than I thought

In [11]:
# Method 2: pull data from pdf
doc = fitz.open("coffeemaker_product.pdf")

In [12]:
# Chunk text from pdf
text_splitter = SemanticChunker(OpenAIEmbeddings())
text_chunks = []
doc_idxs = []

for doc_idx, page in enumerate(doc):
    page_text = page.get_text("text")
    cur_text_chunks = text_splitter.split_text(page_text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

In [13]:
def create_textnode(usepdf=False):
    nodes = []
    # Use scraped chunks
    if usepdf == False:
        # Create a TextNode for the description
        desc_node = TextNode(text=product_data[i]['description'])
        nodes.append(desc_node)

        # Create a TextNode for each review
        for review in product_data[i]['reviews']:
            review_node = TextNode(text=review)
            nodes.append(review_node)
    # Use text from pdf, chunked
    else:
        for idx, text_chunk in enumerate(text_chunks):
            node = TextNode(text=text_chunk)
            src_doc_idx = doc_idxs[idx]
            src_page = doc[src_doc_idx]
            nodes.append(node)
    
    return nodes

In [14]:
### CHANGE HERE ###
# Select which product to analyze
# Madi: Product B and usepdf=False
# Jyoti: usepdf=True

i = "Product A"

nodes = create_textnode(usepdf=False)

In [15]:
gpt_3_5_turbo = "gpt-3.5-turbo"
llm = OpenAI(model=gpt_3_5_turbo)

extractors = [
    TitleExtractor(nodes=5, llm=llm),
    QuestionsAnsweredExtractor(questions=3, llm=llm),
]

pipeline = IngestionPipeline(
    transformations=extractors,
)

nodes = await pipeline.arun(nodes=nodes, in_place=False)

100%|██████████| 101/101 [00:48<00:00,  2.09it/s]


In [16]:
embed_model_3_small = "text-embedding-3-small"
embed_model = OpenAIEmbedding(model=embed_model_3_small, openai_api_key=openai_key)

# create embeddings for the text dataset
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [17]:
use_serverless = os.environ.get("USE_SERVERLESS", "False").lower() == "true"

if use_serverless:
    spec = pinecone.ServerlessSpec(cloud='aws', region='us-west-2')
else:
    spec = pinecone.PodSpec(environment=environment)

# Name our Pinecone Index
index_name = "hw03"

# If a Pinecone index of the same name already exists, delete it:
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

In [18]:
print("Using serverless:",use_serverless)

Using serverless: False


In [19]:
# Name our Pinecone Index
index_name = "hw03"

dimensions = 1536 #768              # the dimensions of the index need to align with the LLM we are using for the RAG system. For example, if using openAI then dimenion = 1536. If using Llama2, then dimension = 384.

pc.create_index(
    name=index_name, 
    dimension=dimensions, 
    metric="cosine",          # we can use different distance metrics to measure the similarity between vector embeddings and user queries. this is where we define what similarity metric we are going to use for the vector store.
    spec=spec
)

# wait for index to be ready before connecting
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

for index in pc.list_indexes():
    print(index['name'])

pc.describe_index("hw03")

pc_index = pc.Index(index_name)  # create an index to use in the vector store

vector_store = PineconeVectorStore(pinecone_index=pc_index)    # this function creates a vector store where we will add and store embeddings

hw03


In [20]:
# nothing so far - empty index
pc_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [21]:
vector_store.add(nodes)

Upserted vectors: 100%|██████████| 101/101 [00:04<00:00, 22.55it/s]


['ff724ff6-f7e9-48af-8efd-c47701e61958',
 'ae318551-e5e6-4f4d-addf-cacb6130274d',
 'ac1e9c3a-7989-4ec8-85b7-5a4f0d7d410a',
 '436f78b6-ac67-4e23-85cb-52cdcd51faca',
 'd0e335e2-b5e8-43ac-a338-f810dc0f9c8e',
 '9392df8d-e2d9-41c1-964c-58d401a064fd',
 '74c88181-0abe-4e0f-accb-e90a3ad59b5d',
 'e13cec00-4207-42c2-a1a6-8ab7a8b25bfb',
 'be6e1e6d-707d-4c03-b135-c93f172b4e01',
 '21719cea-e1d6-4b77-9790-239c957ea07d',
 'd4d63f75-df11-4844-ae5d-737ddeeddcff',
 '058449d2-098c-491d-a41c-581c963964b0',
 '620371c7-fd64-4508-b002-a37089e769c6',
 'bbe60e4d-8a78-4d38-8658-9e2ccb94f4ac',
 'bdb260df-deeb-40ae-9eaf-c86ed22da1f5',
 '1dec90ac-5738-413d-b6e7-d67003bf243c',
 'b90795cc-38d1-4638-9b25-92fd88da8d35',
 '923cc31f-2e4d-4539-bb07-6e920fff9e71',
 '28d82eed-0384-4bc4-9f81-81b7867e6ebe',
 '3002a365-3657-404c-b9fa-0d431323fa62',
 'c4515668-18c9-4343-8cad-6d389f3e3995',
 '6557784c-239f-4cfc-a984-4127e69984b2',
 '3448ecdc-5728-40e3-8f6d-2055c58a936b',
 '4fe83cb3-894a-4658-89aa-da2997ce4049',
 '0cbbee09-e22e-

In [22]:
pc_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [23]:
print(nodes[0].metadata)
print(nodes[0])

{'document_title': '"Vibrant Einstein Canvas Print: A Stylish and High-Quality Addition to Your Home Decor"', 'questions_this_excerpt_can_answer': '1. What type of artwork is featured on the Vibrant Einstein Canvas Print?\n2. Where can the Vibrant Einstein Canvas Print be displayed in a home or office setting?\n3. What materials are used in the production of the Vibrant Einstein Canvas Print to ensure high quality and durability?'}
Node ID: ff724ff6-f7e9-48af-8efd-c47701e61958
Text: About this itemGrafitti art decor Einstein wall art painting on
canvas contemporary art for living room high definition giclee prints
artwork for living room bedroom decoration. This is a good choice for
gifts for relatives and friends.is is a good choice for gifts for
relatives and friends.A perfect wall decorations paintings for living
room, b...


In [38]:
from openai import OpenAI
client = OpenAI()

def get_sys_message(q, k):
    # get embedding
    result = client.embeddings.create(
        input=[q],
        model=embed_model_3_small
    )

    # retrieve from Pinecone vector store
    xq = result.data[0].embedding

    # search
    res2 = pc_index.query(vector=xq, top_k=k, include_metadata=True)
    
    context = ""
    for match in res2['matches']:
        dict_obj = json.loads(match['metadata']['_node_content'])
        context += dict_obj['text'] + " "
    context = context.replace("\n", "")

    return context

In [39]:
queries = ["Based on the product description, describe what the product is and its main purpose.",
           "Describe the potential customer of this product and the emotions associated with it.",
           "Describe how does the product look like, focusing on physical and visual features \
            such as size, material, color, odour, taste, weight.",
           "Describe any specific details or elements of the product that make it different \
            from similar products, including any phrases, symbols, or messages",
           "What is most loved and hated about this product?"
]   

In [40]:
# try a query
get_sys_message(q=queries[0], k=20)

'Great product. Looks great Love it , it is exactly as shown. I loved Cute Great product Same the picture Just like the description. I love it Love it!! Looks just like the picture . Super cute !! Beautifl! It is what you think it is, and it happens to be well packaged, has a mouting loop on back, and print is perfect unless you walk around are wearing jewelers loupes....I would absolutley buy again or buy as a gift. Good As described. Came well packaged. Beautiful canvas! Amazing quality and you can feel the appreciation from the company. And as a customer it makes me want to come back again and again Real nice Looks great  Beautiful! Amazing! '

In [41]:
def rag_openAI_gpt(model, q, k, prompt):
    f=get_sys_message(q, k)
    
    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "Instruction: use the information in" + f + "to answer the user's question."},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": f},
        {"role": "user", "content": "What is the answer?"}
    ]
    )
    return response.choices[0].message.content

In [42]:
# Run
analyses = {}
for i in range(len(queries)):
    response = rag_openAI_gpt(model=gpt_3_5_turbo, q=queries[i], k=20, prompt="Product Description and Reviews")
    print(f"<Query: {queries[i]}>")
    print(response)
    analyses[i+1] = response
    print()
analyses

<Query: Based on the product description, describe what the product is and its main purpose.>
Based on the reviews, the product seems to be exactly as described and shown in the picture. Customers have expressed their love for it, mentioning that it is cute, great, beautiful, and looks just like the picture. The product is well-packaged, has a mounting loop on the back, and the print quality is perfect. Customers are pleased with the quality and would buy it again or as a gift.

<Query: Describe the potential customer of this product and the emotions associated with it.>
Based on the reviews and product description, it seems that the artwork is a large, brightly colored painting that brings joy to the customers who purchased it. The quality is praised, with the colors being described as vibrant and true to the pictures online. While one reviewer mentioned that the material felt a bit cheap, the overall consensus is that the artwork exceeded expectations and is worth the purchase. The c

{1: 'Based on the reviews, the product seems to be exactly as described and shown in the picture. Customers have expressed their love for it, mentioning that it is cute, great, beautiful, and looks just like the picture. The product is well-packaged, has a mounting loop on the back, and the print quality is perfect. Customers are pleased with the quality and would buy it again or as a gift.',
 2: 'Based on the reviews and product description, it seems that the artwork is a large, brightly colored painting that brings joy to the customers who purchased it. The quality is praised, with the colors being described as vibrant and true to the pictures online. While one reviewer mentioned that the material felt a bit cheap, the overall consensus is that the artwork exceeded expectations and is worth the purchase. The customers express their love for the piece and some even mention that they would buy it again or as a gift.',
 3: 'Based on the information provided in the reviews, the product s

In [43]:
# Save responses to the log file
logging.info(analyses)

## **Part 3: Image Generation**
- For each product, based on the information extracted from the product description and customer reviews, craft prompts to guide the image generation process effectively.
- Use the OpenAI’s DALLE 2 to generate 3~5 images for each product based on your crafted prompts. Experiment with different prompts and settings to best visualize what you believe is a good illustration of the product based on product description and customer reviews.
- If necessary, iterate on your prompts based on initial results to refine the illustrations.
- Compare AI-generated product images with the actual product images posted in real world. Are they similar or different? In what dimensions? Do you think AI is able to illustrate the products well? Why or why not?
- Provide analyses and explanations of your findings.

In [44]:
# Change keys to dictionary
analyses = {
    "Product Description and Purpose": analyses[1],
    "Potential Customer/Audience": analyses[2],
    "Physical and Visual Features": analyses[3],
    "Specific Details": analyses[4],
    "Product Sentiment": analyses[5]
}

In [45]:
# Craft prompt
prompt = f"""   
        Generate an image of a product using the following information:
        Description: {analyses["Product Description and Purpose"]}. 
        Audience: {analyses["Potential Customer/Audience"]}.
        Physical and Visual: {analyses["Physical and Visual Features"]}. 
        Details: {analyses["Specific Details"]}.
        Sentiment: {analyses["Product Sentiment"]}.
        """

In [46]:
# Generate images using DALL-E 3
response = openai.images.generate(
    model="dall-e-3",
    prompt=prompt,
    size="1024x1024",
    quality="standard",
    n=1,
)

import webbrowser
webbrowser.open(response.data[0].url)

True

In [47]:
# Craft prompt
prompt2 = f"""   
        Generate an image of a product using the following information. Make sure it captures specific elements of the product:
        Description: {analyses["Product Description and Purpose"]}. 
        Audience: {analyses["Potential Customer/Audience"]}.
        Physical and Visual: {analyses["Physical and Visual Features"]}. 
        Details: {analyses["Specific Details"]}.
        Sentiment: {analyses["Product Sentiment"]}.
        """

In [48]:
# Generate images using DALL-E 3
response = openai.images.generate(
    model="dall-e-3",
    prompt=prompt2,
    size="1024x1024",
    quality="standard",
    n=1,
)

import webbrowser
webbrowser.open(response.data[0].url)

True

In [49]:
# Craft prompt
prompt3 = f"""   
        Generate an image of a product using the following information. Focus on emotions associated with the product:
        Description: {analyses["Product Description and Purpose"]}. 
        Audience: {analyses["Potential Customer/Audience"]}.
        Physical and Visual: {analyses["Physical and Visual Features"]}. 
        Details: {analyses["Specific Details"]}.
        Sentiment: {analyses["Product Sentiment"]}.
        """

In [51]:
# Generate images using DALL-E 3
response = openai.images.generate(
    model="dall-e-3",
    prompt=prompt3,
    size="1024x1024",
    quality="standard",
    n=1,
)

import webbrowser
webbrowser.open(response.data[0].url)

True

In [52]:
# Craft prompt
prompt4 = f"""   
        Generate an image of a product using the following information. Make sure to include the phrase or message that is an important element of the product:
        Description: {analyses["Product Description and Purpose"]}. 
        Audience: {analyses["Potential Customer/Audience"]}.
        Physical and Visual: {analyses["Physical and Visual Features"]}. 
        Details: {analyses["Specific Details"]}.
        Sentiment: {analyses["Product Sentiment"]}.
        """

In [53]:
# Generate images using DALL-E 3
response = openai.images.generate(
    model="dall-e-3",
    prompt=prompt4,
    size="1024x1024",
    quality="standard",
    n=1,
)

import webbrowser
webbrowser.open(response.data[0].url)

True

In [54]:
# Craft prompt
prompt5 = f"""   
        Generate an image of a product using the following information. Include Einstein and holding the sign with 'Love is the answer' message, garffiti on the background:
        Description: {analyses["Product Description and Purpose"]}. 
        Audience: {analyses["Potential Customer/Audience"]}.
        Physical and Visual: {analyses["Physical and Visual Features"]}. 
        Details: {analyses["Specific Details"]}.
        Sentiment: {analyses["Product Sentiment"]}.
        """

In [55]:
# Generate images using DALL-E 3
response = openai.images.generate(
    model="dall-e-3",
    prompt=prompt5,
    size="1024x1024",
    quality="standard",
    n=1,
)

import webbrowser
webbrowser.open(response.data[0].url)

True

In [56]:
# Craft prompt
prompt6 = f"""   
        Generate an image of a product using the following information. Include Einstein holding the sign with 'Love is the answer' message:
        Description: {analyses["Product Description and Purpose"]}. 
        Audience: {analyses["Potential Customer/Audience"]}.
        Physical and Visual: {analyses["Physical and Visual Features"]}. 
        Details: {analyses["Specific Details"]}.
        Sentiment: {analyses["Product Sentiment"]}.
        """

In [57]:
# Generate images using DALL-E 3
response = openai.images.generate(
    model="dall-e-3",
    prompt=prompt6,
    size="1024x1024",
    quality="standard",
    n=1,
)

import webbrowser
webbrowser.open(response.data[0].url)

True